In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install datasets
!pip install transformers
!pip install librosa
!pip install evaluate
!pip install jiwer
!pip install -U datasets

In [3]:
import os
import pandas as pd
from datasets import Dataset, Audio, DatasetDict
import torchaudio
from transformers import pipeline
import evaluate
import zipfile
from sklearn.model_selection import train_test_split

In [4]:
audio_dir = "/content/drive/MyDrive/fine_tune_whisper/hindi_3h/GV_Eval_3h/process_audio"
text_csv = "/content/drive/MyDrive/fine_tune_whisper/hindi_3h/GV_Eval_3h/metadata.csv"

In [5]:
import pandas as pd

df_all = pd.read_csv(text_csv)
df_all = df_all.rename(columns={"file_path": "audio"})

In [69]:
# --- STEP 3: TRAIN/VAL/TEST SPLIT ---
train_val, test = train_test_split(df_all, test_size=0.1, random_state=42)
train, val = train_test_split(train_val, test_size=0.1, random_state=42)

In [70]:
# Convert to HuggingFace format
def to_dataset(df):
    ds = Dataset.from_pandas(df[["audio", "text"]], preserve_index=False)
    return ds.cast_column("audio", Audio(sampling_rate=16000))

In [71]:
dataset = DatasetDict({
    "train": to_dataset(train),
    "validation": to_dataset(val),
    "test": to_dataset(test)
})

In [72]:
# Final splits
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
test_dataset  = dataset["test"]


from datasets import Audio

train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000))
valid_dataset = valid_dataset.cast_column("audio", Audio(sampling_rate=16000))
test_dataset  = test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [73]:
def non_empty_text_filter(batch):
    return [t is not None and t.strip() != "" for t in batch["text"]]

train_dataset = train_dataset.filter(non_empty_text_filter, batched=True)
valid_dataset = valid_dataset.filter(non_empty_text_filter, batched=True)
test_dataset  = test_dataset.filter(non_empty_text_filter, batched=True)

Filter:   0%|          | 0/837 [00:00<?, ? examples/s]

Filter:   0%|          | 0/93 [00:00<?, ? examples/s]

Filter:   0%|          | 0/104 [00:00<?, ? examples/s]

In [11]:
from datasets import load_from_disk, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch

# 🔄 Load model and processor
# model_name = "openai/whisper-small"  # You can also use "medium" or "base"

# # Load processor and split it manually
# processor = WhisperProcessor.from_pretrained(model_name)
# feature_extractor = processor.feature_extractor
# tokenizer = processor.tokenizer

from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [12]:
def preprocess(batch):
    batch["input_features"] = feature_extractor(batch["audio"]["array"], sampling_rate=16000).input_features[0]
    batch["labels"] = tokenizer(batch["text"], max_length=448, truncation=True).input_ids
    return batch

# Apply preprocessing
train_dataset = train_dataset.map(preprocess, remove_columns=train_dataset.column_names)
valid_dataset = valid_dataset.map(preprocess, remove_columns=valid_dataset.column_names)

Map:   0%|          | 0/837 [00:00<?, ? examples/s]

Map:   0%|          | 0/93 [00:00<?, ? examples/s]

In [13]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

# model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")
model.generation_config.language = "Hindi"
model.generation_config.task = "transcribe"

# model.config.forced_decoder_ids = None
# model.generation_config.update(
#     language="hindi".lower(),
#     task="transcribe",
#     forced_decoder_ids=None
# )

model.generation_config.forced_decoder_ids = None

# model.generation_config.forced_decoder_ids = None
# forced_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")
# # model.config.forced_decoder_ids = forced_ids

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [15]:
# model.generation_config

In [16]:
# from transformers import WhisperProcessor, WhisperForConditionalGeneration
# import torchaudio
# import torch

# # device = "cuda" if torch.cuda.is_available() else "cpu"
# # processor = WhisperProcessor.from_pretrained("openai/whisper-small")
# # model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)

# # # Force Gujarati transcription
# # forced_ids = processor.get_decoder_prompt_ids(language="gujarati", task="transcribe")

# # Load test audio
# def load_audio(path):
#     waveform, sample_rate = torchaudio.load(path)
#     if sample_rate != 16000:
#         waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)
#     return waveform.squeeze()

# audio = load_audio(test_dataset[1]["audio"]["path"])
# inputs = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(device)

# # Generate
# with torch.no_grad():
#     generated_ids = model.generate(
#         inputs,
#         # forced_decoder_ids=forced_ids  # 👈 critical!
#         max_length=255,
#     )

# # Decode
# transcription = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
# print(transcription)


In [17]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [18]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [19]:
import evaluate

metric = evaluate.load("wer")

In [20]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hindi-model",
    per_device_train_batch_size=1,  # Lowered to fit 15 GB VRAM
    gradient_accumulation_steps=8,  # To simulate effective batch size of 16
    learning_rate=1.25e-5,
    warmup_steps=500,
    num_train_epochs=4,  # Set to 4 epochs
    gradient_checkpointing=True,  # Saves memory
    fp16=True,  # Mixed precision
    eval_strategy="steps",
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    # generation_max_length=225,
    save_steps=100,
    eval_steps=50,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    save_total_limit=3,  # Optional: keep only last 2 checkpoints to save space
    save_strategy="steps", # Save based on steps
    # push_to_hub=True,
)

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [32]:
processor.save_pretrained(training_args.output_dir)

[]

In [33]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import gc, torch
gc.collect()
torch.cuda.empty_cache()


In [34]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
50,1.106400,1.007989,94.777475
100,0.887900,0.809088,72.297911
150,0.729400,0.710114,64.713896
200,0.689400,0.636382,60.036331
250,0.522700,0.595980,58.719346
300,0.510000,0.553438,55.540418
350,0.349100,0.536596,52.906449
400,0.350700,0.517481,51.316985


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 2

TrainOutput(global_step=420, training_loss=0.6492422728311448, metrics={'train_runtime': 4913.1699, 'train_samples_per_second': 0.681, 'train_steps_per_second': 0.085, 'total_flos': 9.6618392027136e+17, 'train_loss': 0.6492422728311448, 'epoch': 4.0})

In [ ]:
from evaluate import load
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
from datasets import Dataset  # assuming you have the Dataset already loaded
import torch



model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained("/content/whisper-small-hindi-model/checkpoint-420").to("cuda" if torch.cuda.is_available() else "cpu")
# model = WhisperForConditionalGeneration.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="hi", task="transcribe")


wer_metric = load("wer")

# Audio loading helper
def load_audio(path):   
    waveform, sample_rate = torchaudio.load(path)
    if sample_rate != 16000:
        waveform = torchaudio.functional.resample(waveform, sample_rate, 16000)
    return waveform.squeeze()

# Mapping function to generate predictions
def map_to_prediction(batch):
    audio = load_audio(batch["audio"]["path"])
    if len(audio.shape) == 2:
        audio = audio[0]
    inputs = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(inputs, max_length=225)

    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    batch["prediction"] = transcription
    return batch


In [ ]:
# Apply to test set
test_dataset = test_dataset.map(map_to_prediction)

# Calculate WER
wer_score = wer_metric.compute(predictions=test_dataset["prediction"], references=test_dataset["text"])
print(f"\n🔍 WER on test dataset: {wer_score:.2%}")


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

In [49]:
# Apply to test set
test_dataset = test_dataset.map(map_to_prediction)

# Calculate WER
wer_score = wer_metric.compute(predictions=test_dataset["prediction"], references=test_dataset["text"])
print(f"\n🔍 WER on test dataset: {wer_score:.2%}")


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.



🔍 WER on test dataset: 114.75%


In [66]:
import shutil

# Replace 'my_folder' with your folder name
source_folder = '/content/whisper-small-hindi-model/checkpoint-420'
destination_folder = '/content/drive/MyDrive/fine_tune_whisper/whisper-hindi-custom-model-epoch-4'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

'/content/drive/MyDrive/fine_tune_whisper/whisper-hindi-custom-model-epoch-4'